In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
data = pd.read_excel('Example files/EN651 PAH waters export.xlsx')
recovorys = pd.read_excel('Example files/PAH EN651 Recovery Check.xlsx')

In [17]:
data_IS = data[data['Compound'].str.startswith('IS')].copy()
data_IS['rec_norm'] = data_IS['Area']/data_IS['IS Area']

In [18]:
rec_mean = recovorys.dropna(subset=['Area']).groupby('Compound')[['Area','IS Area']].mean().reset_index()
rec_mean['inj_rec_norm'] = rec_mean['Area']/rec_mean['IS Area']

In [19]:
data_IS_all = pd.merge(data_IS,rec_mean,on='Compound')
data_IS_all['Recovory'] = data_IS_all['rec_norm']/data_IS_all['inj_rec_norm']

In [20]:
Recovory_pivot = data_IS_all.pivot(index=['Name','Sample Text'], columns='Compound', values='Recovory').reset_index()

In [22]:
wb = load_workbook('Recoveries.xlsx')
ws = wb.active

In [21]:
Recovory_pivot.to_excel('Recoveries.xlsx', index=False)

Create a new Excel workbook and add a worksheet

In [21]:
Recovory_pivot.to_excel('Recoveries.xlsx', index=False)

In [30]:
wb = Workbook()
ws = wb.active
ws.title = "Recoveries"

Add the DataFrame to the worksheet

In [31]:
for r_idx, row in enumerate(dataframe_to_rows(Recovory_pivot, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

Get the dimensions of the DataFrame (excluding header)

In [32]:
max_row = Recovory_pivot.shape[0] + 1  # Adding 1 because Excel rows start at 1 and the first row is a header
max_col = Recovory_pivot.shape[1]  # The number of columns

Loop through each column (except the first one) and apply the conditional formatting

In [33]:
for col in range(2, max_col + 1):  # Starting from the second column (index 2)
    col_letter = ws.cell(row=1, column=col).column_letter
    cell_range = f'{col_letter}2:{col_letter}{max_row}'  # Exclude the first row (header)

    # Apply a color scale for the values between 0 and 1
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',  # Red color for the minimum value (0)
        mid_type='num', mid_value=0.5, mid_color='FFFF00',  # Yellow for the value 0.5
        end_type='num', end_value=1, end_color='00FF00'  # Green color for the maximum value (1)
    )

    # Apply the color scale rule to the current column
    ws.conditional_formatting.add(cell_range, color_scale_rule)

Calculate the average and standard deviation for each column excluding the first 2 rows as they are labels

In [39]:
average_row = Recovory_pivot.iloc[:, 2:].mean().round(2)
std_dev_row = Recovory_pivot.iloc[:, 2:].std().round(2)

Append the average and standard deviation row at the bottom

In [40]:
avg_row_idx = max_row + 2  # The row index for the average row
ws.cell(row=avg_row_idx, column=1, value="Average")
for c_idx, value in enumerate(average_row, 3):  # Start from the third column
    ws.cell(row=avg_row_idx, column=c_idx, value=value)

std_row_idx = max_row + 3  # The row index for the standard deviation row
ws.cell(row=std_row_idx, column=1, value="Std Dev")
for c_idx, value in enumerate(std_dev_row, 3):  # Start from the third column
    ws.cell(row=std_row_idx, column=c_idx, value=value)

In [41]:
wb.save('Recoveries_Formatted.xlsx')